In [2]:
from sklearn import svm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 8 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
cant_brand_listing             38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
proprosion                     38829 non-null float64
dtypes: float64(7), int64(1)
memory usage: 2.7+ MB


In [4]:
df2 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')
#df3 = pd.read_csv('data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('data/features_checkout.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('data/features_time.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('data/paises_features.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('data/city_features.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('data/sku.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('data/static.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('data/events_fe.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2).fillna(0)
#df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
df_unidos = df_unidos.join(df5).fillna(0)
df_unidos = df_unidos.join(df6).fillna(0)
#df_unidos = df_unidos.join(df7).fillna(0)
#df_unidos = df_unidos.join(df8).fillna(0)
df_unidos = df_unidos.join(df9).fillna(0)
#df_unidos = df_unidos.join(df10).fillna(0)
#df11 =df11.drop("Unnamed: 0",1)
df_unidos = df_unidos.join(df11).fillna(0)
df_unidos = df_unidos.join(df12).fillna(0)

df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 42 columns):
cant_conversions                                38829 non-null float64
cant_checkouts                                  38829 non-null float64
cant_viewed_product                             38829 non-null float64
cant_searched_product                           38829 non-null float64
cant_brand_listing                              38829 non-null float64
total_sesiones                                  38829 non-null int64
promedio_eventos_por_sesion                     38829 non-null float64
proprosion                                      38829 non-null float64
mas_prom                                        38829 non-null float64
vio_mas_apple                                   38829 non-null float64
vio_mas_samsung                                 38829 non-null float64
vio_mas_motorola                                38829 non-null float64
compro_apple                        

In [5]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')


df_test = df_unidos.join(labels, how= "inner")
df_test.head()

,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_brand_listing,total_sesiones,promedio_eventos_por_sesion,proprosion,mas_prom,vio_mas_apple,...,event_checkout,event_conversion,event_generic listing,event_lead,event_search engine hit,event_searched products,event_staticpage,event_viewed product,event_visited site,label
person,,,,,,,,,,,,,,,,,,,,,
ad93850f,0.0,1.0,20.0,0.0,15.0,5,13.000000,0.050000,0.0,1.0,...,0,0,1,0,0,0,0,0,0,0
1b9f7cf6,0.0,1.0,9.0,8.0,3.0,3,8.333333,0.111111,0.0,1.0,...,0,0,0,0,0,1,0,0,0,0
de8fe91b,0.0,1.0,27.0,13.0,3.0,2,26.500000,0.037037,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
45baf068,0.0,1.0,6.0,0.0,3.0,1,20.000000,0.166667,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
99abca5a,2.0,18.0,630.0,5.0,30.0,49,16.714286,0.028571,1.0,0.0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
X, y = df_test.iloc[:,:-1],df_test.iloc[:,-1]

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [9]:
model = svm.SVC()
model.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [11]:
score = model.score(X_test,y_test)
print(score)

0.957249549318


In [13]:
y_final = model.predict(X_test)
y_final

array([0, 0, 0, ..., 0, 0, 0])

In [20]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')


y_pred = []
for y in y_final:
    y_pred.append(y)
roc_auc_score(y_test,y_pred)

kaggle_pred = model.predict(df_events)
#proba_comprar = [x[1] for x in kaggle_pred]
#series = pd.Series(proba_comprar)
#df_submit['label'] = series.values
print(kaggle_pred)
#df_submit.to_csv('submit_svm.csv')

[0 0 0 ..., 0 0 0]
